In [80]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [81]:
news = pd.read_csv('/Users/mackdelany/Documents/DSR Notebooks/NLP-stock-price-prediction/data/train/RedditNews_train.csv')





In [82]:
print(news.shape)

(58908, 2)


In [83]:
news.dropna(inplace=True)

In [84]:
news.shape

(56097, 2)

In [85]:
news['News'] = news['News'].apply(lambda x: x.lower())

In [86]:
news = news.groupby(['Date'])['News'].apply(lambda x: ', '.join(x)).reset_index()

In [87]:
corpus = news['News'].values

In [88]:
transformer = TfidfVectorizer()

In [89]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=200)

In [90]:
news_vectors = vectorizer.fit_transform(corpus).toarray()

In [91]:
news.shape

(2355, 2)

In [92]:
X.toarray().shape

(2355, 200)

In [93]:
pd.DataFrame(news_vectors).shape

(2355, 200)

In [94]:
pd.concat([news.Date, pd.DataFrame(news_vectors)], axis=1)

,Date,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,2008-06-08,0.087215,0.000000,0.000000,0.000000,0.000000,0.000000,0.119453,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.106088,0.000000,0.072767,0.000000,0.088168
1,2008-06-09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.097870,0.000000,0.000000,0.000000,0.000000
2,2008-06-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.128819,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.068816,0.241812,0.000000
3,2008-06-11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.145991,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.158046,0.092559,0.000000
4,2008-06-12,0.000000,0.122114,0.000000,0.000000,0.000000,0.000000,0.119103,0.000000,0.290131,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.105776,0.000000,0.000000,0.000000,0.000000
5,2008-06-13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.105242,0.000000,0.128183,...,0.000000,0.000000,0.113492,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077679
6,2008-06-14,0.085786,0.000000,0.000000,0.000000,0.000000,0.000000,0.117496,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.313049,0.000000,0.071575,0.083835,0.086724
7,2008-06-15,0.084017,0.000000,0.000000,0.000000,0.000000,0.000000,0.345221,0.000000,0.000000,...,0.134224,0.000000,0.000000,0.00000,0.114908,0.000000,0.000000,0.210298,0.000000,0.000000
8,2008-06-16,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.180586,0.000000,0.000000
9,2008-06-17,0.096234,0.000000,0.000000,0.150034,0.000000,0.000000,0.131806,0.000000,0.000000,...,0.000000,0.000000,0.142138,0.00000,0.000000,0.000000,0.000000,0.321167,0.000000,0.000000
